In [1]:
import os
import random
import trimesh
import numpy as np
import mujoco
import mujoco_viewer
import glob
import math
from lxml import etree

# Function swaps attributes in the .xml using lxml etree
def swap_par(tree, element_type, element_name, attribute_name, new_value):   
    element = tree.find(f'.//{element_type}[@name="{element_name}"]')
    element.set(attribute_name, new_value)

f = "DP-Flex_closed_kinematics.xml"

model = mujoco.MjModel.from_xml_path(f)
data = mujoco.MjData(model)

id_JLA = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_left_abduction_p")
id_JRA = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_right_abduction_p")
id_JTA = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_thumb_abduction_p")

id_JTR = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_thumb_rotation_p")

id_JLF_clos = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_left_dynamixel_crank_p")
id_JRF_clos = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_right_dynamixel_crank_p")
id_JTF_clos = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_thumb_dynamixel_crank_p")

viewer = mujoco_viewer.MujocoViewer(model, 
                                    data, 
                                    title="HO", 
                                    width=2560, 
                                    height=1440)

while viewer.is_alive:
    
    # abductions control
    data.ctrl[id_JLA] = np.deg2rad(0) #0..90
    data.ctrl[id_JRA] = np.deg2rad(45) #0..90
    data.ctrl[id_JTA] = np.deg2rad(0) #0..90
       
    # thumb rotation control
    data.ctrl[id_JTR] = np.deg2rad(30) #-90..90

    # closed chain flexion control
    data.ctrl[id_JLF_clos] = np.deg2rad(45) #0..90
    data.ctrl[id_JRF_clos] = np.deg2rad(90) #0..90
    data.ctrl[id_JTF_clos] = np.deg2rad(0) #0..90
    
    mujoco.mj_step(model, data)

    viewer.render()

viewer.close()